## Online Python tutorials

* Basics: [en](https://docs.python.org/3/tutorial) / [de](https://py-tutorial-de.readthedocs.io/de/python-3.3)
* [Numpy tutorial](https://colab.research.google.com/github/cs231n/cs231n.github.io/blob/master/python-colab.ipynb)

## 1. Display random walk data

A random walk adds a (small) random vector to the current state $x(t)$ like this: $x(t+1) = x(t) + \eta$, where $\eta$ is just a random variable, e.g. uniformly distributed in range $[-1, 1]$.

1. Generate a 1d random walk time series, starting from $x(0) = 0$ and display the series with matplotlib (as $x(t)$).
2. Generate a 2d random walk time series, starting from $x(0) = 0$ and display the series with matplotlib (as $x_2(x_1)$).

Avoid using any manual loops (`for` or `while`), but use numpy array functions instead. For example, look for [`cumsum`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cumsum.html).

In [0]:
import numpy as np
import matplotlib.pyplot as plt

eta = np.random.uniform(-1.0, 1.0, size=(100,))


## 2. More Plotting

1. Display the two-dimensional manifold $z=x \cdot y$ as a [wireframe and surface plot](https://matplotlib.org/stable/tutorials/toolkits/mplot3d.html#toolkit-mplot3d-tutorial).
Sample the data in the range $[-1,1]^2$ with 20 samples in each direction.

2. Randomly select 30 images from the MNIST dataset and display them in a $5 \times 6$ grid using subplots.

Again, avoid explicit loops as far as possible.

## 3. Tensors

1. Split the MNIST data set into 120 batches á 500 samples and arrange them as a 4d tensor with the shape (120, 500, 28, 28).

2. Create a new shuffling of these batches.

## 4. Broadcast Operations
In order to perform polynomial fitting we will need to collect the powers $x^1, x^2, x^3, x^4$ of our data samples x in a design matrix A with

$A_{ij} = x_i^j$
 
Create this matrix employing numpy's vectorization and broadcasting capabilities. Don't write an explicit loop.

In [0]:
x = np.arange(-1,1,0.1)

## 5. Timing

Use [timeit](https://docs.python.org/3.8/library/timeit.html) to compare the computation time required for `naive_add` introduced [here](https://colab.research.google.com/github/rhaschke/Neural-Networks/blob/master/Intro%20Colab%2BTensors.ipynb) and numpy's vector-based *add*. For meaningful results, consider random matrices of size larger than $1000^2$.